In [ ]:
# gradio_deployment.ipynb
!pip install gradio tensorflow

In [ ]:
import gradio as gr
import tensorflow as tf
import numpy as np
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Load resources
model = tf.keras.models.load_model('model/machine_translation.keras')

TOKENIZER_EN_PATH = "model/tokenizer_en.pkl"
TOKENIZER_FR_PATH = "model/tokenizer_fr.pkl"


with open(TOKENIZER_EN_PATH, 'rb') as f:
    tokenizer_en = pickle.load(f)
with open(TOKENIZER_FR_PATH, 'rb') as f:
    tokenizer_fr = pickle.load(f)

MAX_LENGTH = 14

def translate(text):
    # Preprocess input
    sequence = tokenizer_en.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=MAX_LENGTH, padding='post')
    
    # Initialize decoder
    decoder_input = np.zeros((1, MAX_LENGTH))
    
    # Generate translation
    for i in range(MAX_LENGTH):
        pred = model.predict([padded, decoder_input], verbose=0)
        predicted_token = np.argmax(pred[0, i])

        decoder_input[0, i] = predicted_token
    translated_sentence = ' '.join([tokenizer_fr.index_word[token] for token in decoder_input[0] if token != 0])
    
    return translated_sentence

# Create Gradio interface
interface = gr.Interface(
    fn=translate,
    inputs=gr.Textbox(label="English Input"),
    outputs=gr.Textbox(label="French Translation"),
    title="English to French Translator",
    examples=[["Hello world"], ["How are you today?"]]
)

interface.launch(share=True)